# Package

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from nltk.metrics.distance import jaro_winkler_similarity
from scipy.spatial.distance import cdist
import unidecode

import zipfile
from urllib.request import urlopen
import shutil
import os

# Dictionnaire départements

In [2]:
DEPARTMENTS = {
    '01': 'Ain', 
    '02': 'Aisne', 
    '03': 'Allier', 
    '04': 'Alpes-de-Haute-Provence', 
    '05': 'Hautes-Alpes',
    '06': 'Alpes-Maritimes', 
    '07': 'Ardèche', 
    '08': 'Ardennes', 
    '09': 'Ariège', 
    '10': 'Aube', 
    '11': 'Aude',
    '12': 'Aveyron', 
    '13': 'Bouches-du-Rhône', 
    '14': 'Calvados', 
    '15': 'Cantal', 
    '16': 'Charente',
    '17': 'Charente-Maritime', 
    '18': 'Cher', 
    '19': 'Corrèze', 
    '2A': 'Corse-du-Sud', 
    '2B': 'Haute-Corse',
    '21': 'Côte-d\'Or', 
    '22': 'Côtes-d\'Armor', 
    '23': 'Creuse', 
    '24': 'Dordogne', 
    '25': 'Doubs', 
    '26': 'Drôme',
    '27': 'Eure', 
    '28': 'Eure-et-Loir', 
    '29': 'Finistère', 
    '30': 'Gard', 
    '31': 'Haute-Garonne', 
    '32': 'Gers',
    '33': 'Gironde', 
    '34': 'Hérault', 
    '35': 'Ille-et-Vilaine', 
    '36': 'Indre', 
    '37': 'Indre-et-Loire',
    '38': 'Isère', 
    '39': 'Jura', 
    '40': 'Landes', 
    '41': 'Loir-et-Cher', 
    '42': 'Loire', 
    '43': 'Haute-Loire',
    '44': 'Loire-Atlantique', 
    '45': 'Loiret', 
    '46': 'Lot', 
    '47': 'Lot-et-Garonne', 
    '48': 'Lozère',
    '49': 'Maine-et-Loire', 
    '50': 'Manche', 
    '51': 'Marne', 
    '52': 'Haute-Marne', 
    '53': 'Mayenne',
    '54': 'Meurthe-et-Moselle', 
    '55': 'Meuse', 
    '56': 'Morbihan', 
    '57': 'Moselle', 
    '58': 'Nièvre', 
    '59': 'Nord',
    '60': 'Oise', 
    '61': 'Orne', 
    '62': 'Pas-de-Calais', 
    '63': 'Puy-de-Dôme', 
    '64': 'Pyrénées-Atlantiques',
    '65': 'Hautes-Pyrénées', 
    '66': 'Pyrénées-Orientales', 
    '67': 'Bas-Rhin', 
    '68': 'Haut-Rhin', 
    '69': 'Rhône',
    '70': 'Haute-Saône', 
    '71': 'Saône-et-Loire', 
    '72': 'Sarthe', 
    '73': 'Savoie', 
    '74': 'Haute-Savoie',
    '75': 'Paris', 
    '76': 'Seine-Maritime', 
    '77': 'Seine-et-Marne', 
    '78': 'Yvelines', 
    '79': 'Deux-Sèvres',
    '80': 'Somme', 
    '81': 'Tarn', 
    '82': 'Tarn-et-Garonne', 
    '83': 'Var', 
    '84': 'Vaucluse', 
    '85': 'Vendée',
    '86': 'Vienne', 
    '87': 'Haute-Vienne', 
    '88': 'Vosges', 
    '89': 'Yonne', 
    '90': 'Territoire de Belfort',
    '91': 'Essonne', 
    '92': 'Hauts-de-Seine', 
    '93': 'Seine-Saint-Denis', 
    '94': 'Val-de-Marne', 
    '95': 'Val-d\'Oise',
    '971': 'Guadeloupe', 
    '972': 'Martinique', 
    '973': 'Guyane', 
    '974': 'La Réunion',
    '975': 'Saint-Pierre et Miquelon',
    '976': 'Mayotte',
}

flipped_dict = dict(zip(DEPARTMENTS.values(), DEPARTMENTS.keys()))

# Taux de natalité

Fichiers naissance par département
Source https://www.insee.fr/fr/statistiques/2540004?sommaire=4767262 
Puis Fichiers par départements de naissance, csv 12 mo

In [3]:
link = "https://www.insee.fr/fr/statistiques/fichier/2012761/TCRD_053.xlsx"
df_natalite = pd.read_excel(link,"DEP", header = 3, skipfooter = 2, 
                                 names = np.array(["code_dep", "nom_dep", "tx_natalite_2020_percent"]),
                                usecols = "A,B,C")

df_natalite.sample(n = 10)

,code_dep,nom_dep,tx_natalite_2020_percent
25,25,Doubs,10.5
15,16,Charente,8.4
5,06,Alpes-Maritimes,10.3
24,24,Dordogne,7.3
92,92,Hauts-de-Seine,13.2
14,15,Cantal,7.2
79,79,Deux-Sèvres,8.4
34,34,Hérault,10.4
58,58,Nièvre,7.4
48,48,Lozère,6.9


# Musées

Liste et localisation des musées en France
https://www.data.gouv.fr/fr/datasets/liste-et-localisation-des-musees-de-france/
Puis liste des musées de france au 31/12/2017 (238.3ko)

In [4]:
musees = pd.read_excel("https://www.data.gouv.fr/fr/datasets/r/22df4a13-72d8-4b34-940e-8aec297b5ded",
                      usecols = "A,B,E:I")

In [5]:
musees.head()

,NEW REGIONS,NOMDEP,ID MUSEE,NOM DU MUSEE,ADR,CP,VILLE
0,AUVERGNE-RHÔNE-ALPES,AIN,0105302,Musée Départemental des Pays De l'Ain,"34, rue du Général Delestraint",1000.0,BOURG-EN-BRESSE
1,AUVERGNE-RHÔNE-ALPES,AIN,0105301,Musée du Brou,"Monastère Royal de Brou\n63, Boulevard de Brou",1000.0,BOURG-EN-BRESSE
2,AUVERGNE-RHÔNE-ALPES,AIN,0106401,Musée de la Société d'Histoire et d'Archéologie,Mairie,1470.0,BRIORD
3,AUVERGNE-RHÔNE-ALPES,AIN,0119201,Musée Archéologique,Place de l'Eglise,1580.0,IZERNORE
4,AUVERGNE-RHÔNE-ALPES,AIN,0121801,Musée du Bugey-Valmorey,Maison Renaissance,1260.0,LOCHIEU


In [6]:
df_musees = musees.groupby("NOMDEP").size().reset_index()

df_musees = df_musees.rename(columns = {"NOMDEP":"nom_dep", 0:"nb_musees"})
display(df_musees)
display(df_musees.loc[81])
#df_musees.columns = [["nom_dep", "nb_musees"]]

,nom_dep,nb_musees
0,AIN,14
1,AISNE,15
2,ALLIER,9
3,ALPES DE HAUTE PROVENCE,9
4,ALPES-MARITIMES,33
...,...,...
96,VENDEE,14
97,VIENNE,12
98,VOSGES,7
99,YONNE,12


nom_dep      SAINT-PIERRE ET MIQUELON
nb_musees                           1
Name: 81, dtype: object

In [7]:
# Création d'un array à partir des clés du dictionnaire qui donne nom_departement --> numero
arr_dic = np.array([unidecode.unidecode(word.lower()) for word in flipped_dict.keys()])

# Recréation du dictionnaire avec des clés dans le même format que pour l'array au dessus
dic_dpt_lower = {}

for key, values in flipped_dict.items():
    dic_dpt_lower.update({unidecode.unidecode(key.lower()):values})

#display(dic_dpt_lower)

In [8]:
# Création d'un dictionnaire à partir des noms des départements dans notre dataframe sur les musées
arr_dpt_mus = np.array(df_musees["nom_dep"].str.lower())
#display(arr_dpt_mus)

In [9]:
# Cellule utilisée pour faire des tests concernant la similarité de jaro et winkler
# Noter que la casse (majuscule / minuscule) est importante pour la similarité utilisée

x = str(arr_dic[0])
y = arr_dpt_mus[0]
display(x)
display(type(x))
display(y)
display(type(y))

display("-----")
display(jaro_winkler_similarity("boh","boh"))
display(jaro_winkler_similarity(x,y))

'ain'

str

'ain'

str

'-----'

1.0

1.0

In [10]:
# Création d'une matrice avec pour chaque case la similarité entre le nom du département venant du df musées (en ligne)
# et celui venant de notre dictionnaire (en colonnes, donc arr_dic est en colonnes)

matrix = cdist(arr_dpt_mus.reshape(-1, 1), arr_dic.reshape(-1, 1), lambda x, y: jaro_winkler_similarity(x[0], y[0]))
#display(matrix.shape)

# Puisqu'on a un score de similarité, on garde pour chaque ligne l'index de la valeur la plus élevée
correct = np.argmax(matrix, axis = 1)

#Cette boucle permet de vérifier que les associations sont correctes :
#for i in range(len(correct)):
#    print(arr_dic[correct[i]], arr_dpt_mus[i])
#    print("---")

In [11]:
# Ajout de la colonne qui nous intéresse dans df_musees
df_musees["code_dep"] = " "

for i in range(df_musees.shape[0]):
    """
    Peu élégant, mais fonctionnel, il faudrait réfléchir un peu pour utiliser amap ou apply
    """
    # Les deux variables ci-dessous ont pour seule utilité de rendre le code un tout petit peu plus lisible
    val_inter = list(arr_dpt_mus).index(df_musees["nom_dep"][i].lower())
    code_dep = dic_dpt_lower[arr_dic[correct[val_inter]]]
    df_musees["code_dep"][i] = code_dep

C:\Users\leo\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [12]:
display(df_musees)

,nom_dep,nb_musees,code_dep
0,AIN,14,01
1,AISNE,15,02
2,ALLIER,9,03
3,ALPES DE HAUTE PROVENCE,9,04
4,ALPES-MARITIMES,33,06
...,...,...,...
96,VENDEE,14,85
97,VIENNE,12,86
98,VOSGES,7,88
99,YONNE,12,89


# Chômage 

In [13]:
# importation des données 

url = "https://www.insee.fr/fr/statistiques/series/csv/famille/102760732"
file_name = 'famille_TAUX-CHOMAGE_23122021.zip'

# extracting zipfile from URL
with urlopen(url) as response, open(file_name, 'wb') as out_file:
   shutil.copyfileobj(response, out_file)

# extracting required file from zipfile
with zipfile.ZipFile(file_name) as zf:
   zf.extract('valeurs_trimestrielles.csv')

# deleting the zipfile from the directory
os.remove('famille_TAUX-CHOMAGE_23122021.zip')

# loading data from the file
chomage = pd.read_csv('valeurs_trimestrielles.csv', sep=";")
chomage.head()
#display("Libellés d'origine")
#display(chomage["Libellé"].unique())

# on fait un peu de nettoyage
chomage_vf = chomage.drop(chomage[chomage["Libellé"] == "Codes"].index)
chomage_vf = chomage_vf.reset_index()
chomage_vf.shape

chomage_vf["Libellé"] = [nom.split(" ")[7] for nom in chomage_vf["Libellé"]]
chomage_vf = chomage_vf.drop(columns = ["index", "idBank", "Dernière mise à jour", "Période"])
#display("Libellés modifiés")
#display(chomage_vf["Libellé"].unique())
#display("Attention, maintenant on a plus dans le nom l'indication que c'est un département ou une région !")

In [14]:
#display(chomage_vf.head())

# On ne garde que les valeurs depuis 2016
liste_cols = ["Libellé"]
annees = ["20"+str(i) for i in range(16,21)]
trimestres = ["-T"+str(i) for i in range(1,5)]

for an in annees:
    for tr in trimestres:
        liste_cols.append(an+tr)
liste_cols.append("2021-T1")
liste_cols.append("2021-T2")
#print(liste_cols)

chomage_vf = chomage_vf.filter(liste_cols, axis=1)
#display(chomage_vf)

chomage_vf = chomage_vf.replace(to_replace = 'La', value = 'La Réunion', regex=True)
chomage_vf = chomage_vf.replace(to_replace = 'Territoire', value = 'Territoire de Belfort', regex=True)
chomage_vf.isna().sum().any() # Pas de NA dans le df a priori

False

In [15]:
# je ne comprends pas pourquoi ça ne fonctionne pas ==> ça met des NaN 
liste_cols = liste_cols[1:]
print(liste_cols)

# Les valeurs étaient des objets, on les transforme en entiers
chomage_vf = chomage_vf.apply(lambda x: x.astype(float) if x.name in liste_cols else x)

for an in annees:
    chomage_vf[an] = chomage_vf[[an+str(tr) for tr in trimestres]].mean(axis = 1)

chomage_vf['2021'] = chomage_vf[["2021-T1","2021-T2"]].mean(axis=1)
display(chomage_vf)

['2016-T1', '2016-T2', '2016-T3', '2016-T4', '2017-T1', '2017-T2', '2017-T3', '2017-T4', '2018-T1', '2018-T2', '2018-T3', '2018-T4', '2019-T1', '2019-T2', '2019-T3', '2019-T4', '2020-T1', '2020-T2', '2020-T3', '2020-T4', '2021-T1', '2021-T2']


,Libellé,2016-T1,2016-T2,2016-T3,2016-T4,2017-T1,2017-T2,2017-T3,2017-T4,2018-T1,...,2020-T3,2020-T4,2021-T1,2021-T2,2016,2017,2018,2019,2020,2021
0,France,9.9,9.8,9.6,9.7,9.3,9.2,9.3,8.7,8.9,...,8.9,7.8,7.8,7.8,9.750,9.125,8.700,8.175,7.850,7.80
1,Île-de-France,8.8,8.7,8.6,8.8,8.3,8.2,8.3,7.9,8.1,...,8.4,7.7,7.7,7.6,8.725,8.175,7.925,7.400,7.350,7.65
2,Centre-Val,9.7,9.5,9.4,9.4,9.0,9.0,8.9,8.4,8.6,...,8.3,7.1,7.2,7.2,9.500,8.825,8.450,8.050,7.400,7.20
3,Pays,8.8,8.5,8.4,8.4,8.0,7.9,7.9,7.3,7.6,...,8.0,6.7,6.8,6.7,8.525,7.775,7.375,7.050,7.000,6.75
4,Bretagne,8.8,8.6,8.4,8.6,8.1,8.0,8.0,7.5,7.7,...,7.7,6.6,6.6,6.5,8.600,7.900,7.500,6.975,6.775,6.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,Var,11.1,10.8,10.7,11.0,10.6,10.4,10.4,9.9,10.2,...,9.7,8.2,8.4,8.6,10.900,10.325,9.925,9.200,8.575,8.50
110,Guadeloupe,25.6,25.1,21.5,20.6,21.6,22.9,21.4,22.5,23.5,...,18.9,19.4,17.5,19.2,23.200,22.100,22.025,20.125,16.850,18.35
111,Martinique,19.8,19.7,19.1,17.4,19.4,19.2,18.3,17.3,16.5,...,16.4,15.0,12.7,14.7,19.000,18.550,18.475,16.150,13.050,13.70
112,Guyane,19.4,19.8,22.8,19.1,18.7,20.4,20.5,18.1,15.8,...,17.1,17.8,12.5,13.8,20.275,19.425,16.825,17.125,14.125,13.15


In [16]:
chomage_vf["Département"] = chomage_vf["Libellé"].map(flipped_dict)
chomage_vf.drop(chomage_vf[chomage_vf['Département'].isnull()].index, inplace=True)

chomage_final = chomage_vf[["Département","Libellé","2016","2017","2018","2019","2020","2021"]].copy()
chomage_final = chomage_final.rename(columns = {"Département": "code_dep", "Libellé": "nom_dep", "2016": "chomage_2016",
                               "2017": "chomage_2017", "2018": "chomage_2018", "2019": "chomage_2019",
                               "2020": "chomage_2020", "2021": "chomage_2019"})


chomage_final = chomage_final.reset_index().drop("index", axis =1).sort_values(by = 'code_dep')
display(chomage_final)
display(chomage_final.shape)

,code_dep,nom_dep,chomage_2016,chomage_2017,chomage_2018,chomage_2019,chomage_2020,chomage_2019
1,01,Ain,7.350,6.775,6.325,6.050,6.075,6.05
2,02,Aisne,13.800,13.175,12.575,11.775,11.125,11.15
3,03,Allier,10.525,9.900,9.525,9.050,8.650,8.45
4,04,Alpes-de-Haute-Provence,11.450,11.075,10.750,10.075,9.400,9.35
5,05,Hautes-Alpes,9.275,8.750,8.475,7.950,7.575,8.40
...,...,...,...,...,...,...,...,...
93,95,Val-d'Oise,10.150,9.575,9.275,8.625,8.575,8.85
95,971,Guadeloupe,23.200,22.100,22.025,20.125,16.850,18.35
96,972,Martinique,19.000,18.550,18.475,16.150,13.050,13.70
97,973,Guyane,20.275,19.425,16.825,17.125,14.125,13.15


(99, 8)

# Taux de logements vacants 

en théorie, plus le taux de logements vacants est élevé dans une zone, plus les prix sont bas 

-Nombre de logements du parc privé
-Nombre de logements vacants du parc privé au 01/01/2019
-Nombre de logements du parc privé vacants depuis moins de deux ans au 01/01/2019
-Nombre de logements du parc privé vacants depuis deux ans ou plus au 01/01/2019
-Taux de logements vacants du parc privé au 01/01/2019
-Taux de logements du parc privé vacants depuis moins de deux ans au 01/01/2019
-Taux de logements du parc privé vacants depuis deux ans ou plus au 01/01/2019

In [17]:
link = 'https://www.data.gouv.fr/fr/datasets/r/d6b298be-42ee-4a9f-9a01-c8cb6edd4978'
vacants = pd.read_excel(link, 'Données')

In [18]:
vacants_new = vacants[["INSEE_COM","CODE_DEPT","NOM_DEPT","Prop_logvac_pp_010119"]]
vacants_new.head()
vacants_new = vacants_new.rename(columns = {"INSEE_COM": "code_postal", "CODE_DEPT": "code_dep", "NOM_DEPT": "nom_dep",
                                           "Prop_logvac_pp_010119": "taux_vacances_2019"})


vacants_new = vacants_new.groupby("nom_dep").mean("taux_vacance_2019").reset_index()
vacants_new["code_dep"] = vacants_new["nom_dep"].map(flipped_dict)
vacants_new = vacants_new.sort_values(by = "code_dep").reset_index().drop(columns = "index")
vacants_new

,nom_dep,taux_vacances_2019,code_dep
0,Ain,10.066116,01
1,Aisne,10.998363,02
2,Allier,14.294594,03
3,Alpes-de-Haute-Provence,11.413979,04
4,Hautes-Alpes,9.437147,05
...,...,...,...
95,Val-d'Oise,8.253955,95
96,Guadeloupe,16.804188,971
97,Martinique,13.315198,972
98,Guyane,15.635261,973


3 facteurs qui influencent les prix de l'immobilier : http://www.bsi-economics.org/61-dynamique-des-prix-de-l%EF%BF%BDimmobilier-quels-determinants

- l'environnement économique et financier (offre et demande de logement, contexte économie, structure des marchés immobiliers, logements neufs et anciens, démographie, taux de logements vacants)

- les conditions d'emprunts (taux variables/fixes, maturités des prêts, ratio emprunt sur valeur, répartition des crédits, solvabilité, système de garantie)

- l'environnement fiscal (mesures fiscales incitatives à la location, distorsion fiscale des locataires vers les propriétaires)


# Jointure des tables

In [19]:
#Antijoin pour regarder ce qui n'a pas matché : parce que la liste des départements devrait être la même partout
#Innerjoin pour éviter les NA 

In [20]:
dep_nata = [i for i in df_natalite["code_dep"]]
dep_musees = [j for j in df_musees["code_dep"]]
dep_chomage = [j for j in chomage_final["code_dep"]]
dep_vac = [j for j in vacants_new["code_dep"]]

# On fait l'intersection successive des départements de chaque df
dep_commun = list(set(dep_nata).intersection(set(dep_musees).intersection(set(dep_chomage).intersection(dep_vac))))
display(len(dep_commun))

99

**Bonne nouvelle : 99 départements en commun et autant dans notre df d'origine !**

In [21]:
not_commun_nata = [i for i in dep_nata if i not in dep_commun]
display(not_commun_nata)

not_commun_musees = [i for i in dep_musees if i not in dep_commun]
display(not_commun_musees)

not_commun_chomage = [i for i in dep_chomage if i not in dep_commun]
display(not_commun_chomage)

not_commun_vac = [i for i in dep_vac if i not in dep_commun]
display(not_commun_vac)

['40', 'M', '976']

['40', '975']

[]

['40']

Les Landes (**40**) sont absentes du df chomage
Saint-Pierre et Miquelon (**975**) et Mayotte (**976**) ne sont pas présents partout 

La nature du **M** n'est pas encore identifiée

In [22]:
display(df_natalite)
display(df_natalite.shape)
type(df_natalite["code_dep"][0])

,code_dep,nom_dep,tx_natalite_2020_percent
0,01,Ain,10.3
1,02,Aisne,10.1
2,03,Allier,7.8
3,04,Alpes-de-Haute-Provence,8.0
4,05,Hautes-Alpes,8.3
...,...,...,...
97,971,Guadeloupe,12.4
98,972,Martinique,9.9
99,973,Guyane,27.5
100,974,La Réunion,15.4


(102, 3)

str

In [23]:
display(df_musees)
display(df_musees.shape)
type(df_musees["code_dep"][0])

,nom_dep,nb_musees,code_dep
0,AIN,14,01
1,AISNE,15,02
2,ALLIER,9,03
3,ALPES DE HAUTE PROVENCE,9,04
4,ALPES-MARITIMES,33,06
...,...,...,...
96,VENDEE,14,85
97,VIENNE,12,86
98,VOSGES,7,88
99,YONNE,12,89


(101, 3)

str

In [24]:
display(chomage_final)
display(chomage_final.shape)

,code_dep,nom_dep,chomage_2016,chomage_2017,chomage_2018,chomage_2019,chomage_2020,chomage_2019
1,01,Ain,7.350,6.775,6.325,6.050,6.075,6.05
2,02,Aisne,13.800,13.175,12.575,11.775,11.125,11.15
3,03,Allier,10.525,9.900,9.525,9.050,8.650,8.45
4,04,Alpes-de-Haute-Provence,11.450,11.075,10.750,10.075,9.400,9.35
5,05,Hautes-Alpes,9.275,8.750,8.475,7.950,7.575,8.40
...,...,...,...,...,...,...,...,...
93,95,Val-d'Oise,10.150,9.575,9.275,8.625,8.575,8.85
95,971,Guadeloupe,23.200,22.100,22.025,20.125,16.850,18.35
96,972,Martinique,19.000,18.550,18.475,16.150,13.050,13.70
97,973,Guyane,20.275,19.425,16.825,17.125,14.125,13.15


(99, 8)

In [25]:
display(vacants_new)
display(vacants_new.shape)

,nom_dep,taux_vacances_2019,code_dep
0,Ain,10.066116,01
1,Aisne,10.998363,02
2,Allier,14.294594,03
3,Alpes-de-Haute-Provence,11.413979,04
4,Hautes-Alpes,9.437147,05
...,...,...,...
95,Val-d'Oise,8.253955,95
96,Guadeloupe,16.804188,971
97,Martinique,13.315198,972
98,Guyane,15.635261,973


(100, 3)

In [51]:
df_commun = (df_natalite.set_index("code_dep")
            .join(df_musees.set_index("code_dep"), how = "inner", rsuffix = "jo")
            .join(chomage_final.set_index("code_dep"), how = "inner", rsuffix = "jo1")
            .join(vacants_new.set_index("code_dep"), how = "inner", rsuffix = "jo2"))
df_commun.sample(n= 10)

,nom_dep,tx_natalite_2020_percent,nom_depjo,nb_musees,nom_depjo1,chomage_2016,chomage_2017,chomage_2018,chomage_2019,chomage_2020,chomage_2019,nom_depjo2,taux_vacances_2019
code_dep,,,,,,,,,,,,,
34,Hérault,10.4,HERAULT,22,Hérault,13.825,13.100,12.650,11.925,11.175,11.15,Hérault,9.496227
39,Jura,8.7,JURA,13,Jura,7.525,6.850,6.425,6.125,5.775,5.45,Jura,12.748080
82,Tarn-et-Garonne,9.4,TARN-ET-GARONNE,6,Tarn-et-Garonne,11.500,10.825,10.375,9.575,9.000,9.20,Tarn-et-Garonne,10.122531
06,Alpes-Maritimes,10.3,ALPES-MARITIMES,33,Alpes-Maritimes,10.400,9.900,9.450,8.825,8.875,8.95,Alpes-Maritimes,10.294761
13,Bouches-du-Rhône,12.1,BOUCHES DU RHONE,44,Bouches-du-Rhône,11.650,11.225,10.825,10.025,9.300,9.25,Bouches-du-Rhône,8.429049
43,Haute-Loire,8.3,HAUTE-LOIRE,5,Haute-Loire,8.275,7.475,7.125,6.625,6.325,6.15,Haute-Loire,13.874871
68,Haut-Rhin,9.9,HAUT-RHIN,24,Haut-Rhin,9.800,9.025,8.350,8.000,8.025,7.95,Haut-Rhin,9.029670
63,Puy-de-Dôme,9.3,PUY DE DÔME,13,Puy-de-Dôme,8.750,8.175,7.825,7.300,6.975,6.90,Puy-de-Dôme,12.674410
93,Seine-Saint-Denis,16.6,SEINE SAINT-DENIS,5,Seine-Saint-Denis,12.825,12.025,11.700,10.925,10.650,11.05,Seine-Saint-Denis,9.285712


En comparant les colonnes nom_dep, nom_depjo1 & nom_depjo2, on constate que la jointure semble avoir fonctionné correctement

In [48]:
df_commun = df_commun.drop(columns = ["nom_depjo", "nom_depjo1", "nom_depjo2"]).reset_index()

In [50]:
display(df_commun.sample(n = 10))
display(df_commun.shape)

,code_dep,nom_dep,tx_natalite_2020_percent,nb_musees,chomage_2016,chomage_2017,chomage_2018,chomage_2019,chomage_2020,chomage_2019,taux_vacances_2019
53,54,Meurthe-et-Moselle,9.3,11,9.850,9.275,8.650,8.000,7.575,7.55,10.287818
89,90,Territoire de Belfort,10.0,4,10.875,9.600,9.150,8.875,8.975,9.00,9.051611
2,03,Allier,7.8,9,10.525,9.900,9.525,9.050,8.650,8.45,14.294594
57,58,Nièvre,7.4,11,9.175,8.225,7.575,7.150,6.625,6.55,13.494511
82,83,Var,9.5,17,10.900,10.325,9.925,9.200,8.575,8.50,8.844246
71,72,Sarthe,9.9,6,9.750,9.000,8.600,8.300,8.125,7.80,9.537642
94,95,Val-d'Oise,15.1,8,10.150,9.575,9.275,8.625,8.575,8.85,8.253955
87,88,Vosges,8.3,7,11.100,10.100,9.500,9.000,8.475,8.25,11.738203
50,51,Marne,10.4,10,9.450,8.750,8.375,7.850,7.600,7.50,10.636940
84,85,Vendée,8.6,14,7.925,7.150,6.750,6.500,6.500,6.10,5.960910


(99, 11)

**On a bien les 99 lignes correspondant aux départements**

In [52]:
def retrieve_data():
    """
    C'est très peu propre mais fonctionnel
    """
    return dep_commun, df_commun